In [2]:
from pymongo import MongoClient
from pprint import pprint
import uuid
import requests
import json

In [6]:
client = MongoClient(host='34.227.105.4:27017',serverSelectionTimeoutMS = 3000)
mydb = client["stargatedb"]
aircol = mydb["airline"]
for x in aircol.find():
    print(x)

{'_id': ObjectId('5f85c824512add3976c183ac'), 'id': 10, 'type': 'airline', 'name': '40-Mile Air', 'iata': 'Q5', 'icao': 'MLA', 'callsign': 'MILE-AIR', 'country': 'United States'}
{'_id': ObjectId('5f85c824512add3976c183ad'), 'id': 10123, 'type': 'airline', 'name': 'Texas Wings', 'iata': 'TQ', 'icao': 'TXW', 'callsign': 'TXW', 'country': 'United States'}
{'_id': ObjectId('5f85c824512add3976c183ae'), 'id': 10226, 'type': 'airline', 'name': 'Atifly', 'iata': 'A1', 'icao': 'A1F', 'callsign': 'atifly', 'country': 'United States'}
{'_id': ObjectId('5f85c824512add3976c183af'), 'id': 10642, 'type': 'airline', 'name': 'Jc royal.britannica', 'iata': '', 'icao': 'JRB', 'callsign': '', 'country': 'United Kingdom'}
{'_id': ObjectId('5f85c824512add3976c183b0'), 'id': 10748, 'type': 'airline', 'name': 'Locair', 'iata': 'ZQ', 'icao': 'LOC', 'callsign': 'LOCAIR', 'country': 'United States'}
{'_id': ObjectId('5f85c824512add3976c183b1'), 'id': 10765, 'type': 'airline', 'name': 'SeaPort Airlines', 'iata':

In [4]:
url = "https://c72c60e1-d1c1-4730-8d6a-413abce921ff-us-east-1.apps.astra.datastax.com/api/rest/v1/auth"

payload = {
    "username": "mongo_migrator",
    "password": "mongo_migrator"
}
headers = {
    "accept": "*/*",
    "x-cassandra-request-id": "134",
    "content-type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"authToken":"dbe36dbf-8d9c-4c65-8a5d-184afc09b51f"}


In [9]:
headers = {
    "x-cassandra-token": "dbe36dbf-8d9c-4c65-8a5d-184afc09b51f",
    "content-type": "application/json"
}

URL = "https://c72c60e1-d1c1-4730-8d6a-413abce921ff-us-east-1.apps.astra.datastax.com"
COLLECTION = "dj_airlines"
NAMESPACE = "mongo_migrator"

for data in aircol.find():
    data.pop('_id')
    AIRLINE_ID = str(uuid.uuid4())
    DOC_ROOT_PATH = f"/api/rest/v2/namespaces/{NAMESPACE}/collections/{COLLECTION}/{AIRLINE_ID}"
    PUTTURL = URL+DOC_ROOT_PATH
    response = requests.request("PUT", PUTTURL, data=json.dumps(data), headers=headers)
    pprint(response.text)

'Unable to create schema for collection'
'{"documentId":"11a3a23c-4129-409a-92da-d49e99166a08"}'
'{"documentId":"d59d7f5c-2724-496b-87a7-54b79aa9a5a8"}'
'{"documentId":"b674248f-bde2-4c0d-b7bb-20134eb73c3d"}'
'{"documentId":"510c756f-e328-453f-b0f1-2ffb50ad453a"}'
'{"documentId":"1abd854d-3639-4957-a63b-e164e68df0f7"}'
'{"documentId":"7f8630bc-f8f2-420f-8fc9-c2e493198588"}'
'{"documentId":"8ce44ac8-a845-419f-94e1-4b866972a887"}'
'{"documentId":"a2bf7509-0d5f-4ad0-bd7f-bd19a5eaa30e"}'
'{"documentId":"aaf277d1-9f52-4164-be90-1111b8602267"}'
'{"documentId":"ff3be73d-69cb-4d96-a30d-25f0d67ea874"}'
'{"documentId":"9e209121-7eae-4abc-b67d-2467f9fa6ac9"}'
'{"documentId":"fec45c52-51f9-4792-99be-1b9856b0e7bb"}'
'{"documentId":"95dd7646-58e5-4633-a47b-153d5470e84e"}'


In [87]:
GET = f"/api/rest/v2/namespaces/{NAMESPACE}/collections/{COLLECTION}/0e884f7c-837f-4c93-9d73-b795e89e8127"
GETTURL = URL+GET
response = requests.request("GET", GETTURL, headers=headers)
pprint(response.text)

('{"documentId":"0e884f7c-837f-4c93-9d73-b795e89e8127","data":{"callsign":"FLYSTAR","country":"United '
 'Kingdom","iata":"5W","icao":"AEU","id":112,"name":"Astraeus","type":"airline"}}')


In [35]:
COLLECTION = "mongo_air"
for i in range(8):
    if i==0:
        for data in aircol.find().limit(1):
            last_id=data.pop('_id')
            AIRLINE_ID = str(uuid.uuid4())
            DOC_ROOT_PATH = f"/api/rest/v2/namespaces/{NAMESPACE}/collections/{COLLECTION}/{AIRLINE_ID}"
            PUTTURL = URL+DOC_ROOT_PATH
            response = requests.request("PUT", PUTTURL, data=json.dumps(data), headers=headers)
            pprint(response.text)
    else:
        for data in aircol.find({'_id': {'$gt': last_id}}).limit(1):
            last_id=data.pop('_id')
            AIRLINE_ID = str(uuid.uuid4())
            DOC_ROOT_PATH = f"/api/rest/v2/namespaces/{NAMESPACE}/collections/{COLLECTION}/{AIRLINE_ID}"
            PUTTURL = URL+DOC_ROOT_PATH
            response = requests.request("PUT", PUTTURL, data=json.dumps(data), headers=headers)
            pprint(response.text)
    print(last_id)

'Unable to create schema for collection'
5f84d15b4472b78f9faae7f8
'{"documentId":"a0deedbc-cae2-440b-9fe8-5c5378a3bb59"}'
5f84d15b4472b78f9faae7f9
'{"documentId":"597f5673-8ba2-4341-9c21-199747e4e6e8"}'
5f84d15b4472b78f9faae7fa
'{"documentId":"47b55399-3cee-4992-a8ad-8a38751841dd"}'
5f84d15b4472b78f9faae7fb
'{"documentId":"9b7f1a69-cbbe-46e6-9e87-c2b85961f76e"}'
5f84d15b4472b78f9faae7fc
'{"documentId":"85decbbf-24c1-4a5a-9a34-3ee7242be472"}'
5f84d15b4472b78f9faae7fd
'{"documentId":"d518a4f1-24d5-4294-a69d-47e77c9288cd"}'
5f84d15b4472b78f9faae7fe
5f84d15b4472b78f9faae7fe


In [50]:
      data = aircol.find().limit(1)
      data.pop('_id')

None
